In [2]:
import os 
import pandas as pd 
import numpy as np 

In [3]:
file_name = 'Maharashtra_AE.csv'
maharashtra_election_df = pd.read_csv(file_name) 

file_name = 'Maharashtra_GA.csv'
maharashtra_ge_election_df = pd.read_csv(file_name) 

/tmp/ipykernel_204337/2797126938.py:2: DtypeWarning: Columns (13,18,19,38,39,40,41,42,43,44,45) have mixed types. Specify dtype option on import or set low_memory=False.
  maharashtra_election_df = pd.read_csv(file_name)


In [4]:
excel_dir = 'intermediate_tables/MH'
years = ['2014', '2019']  # Define the years you want to include
excel_files = [] 
ACS = 288


In [5]:
import os
import pandas as pd
from functools import reduce

def generate_swing_and_strong_booths_df(AC, years, excel_dir):
    # Prepare to collect the file paths
    excel_files = []
    for dirpath, dirnames, filenames in os.walk(excel_dir):
        for year in years:
            if os.path.basename(dirpath) == year:  # Checking directory names against the specified years
                for filename in filenames:
                    if filename == f"{AC}.xlsx":  # Looking for specific Excel files
                        full_path = os.path.join(dirpath, filename)
                        excel_files.append(full_path)

    # Reading and merging Excel files
    merged_dfs = []
    for excel_file in excel_files:
        df = pd.read_excel(excel_file)
        year = os.path.basename(os.path.dirname(excel_file))  # Assuming folder names are the years
        if 'INC_Status' in df.columns:
            df.rename(columns={'INC_Status': year}, inplace=True)
        df.dropna(subset=['SN'], inplace=True)
        df['SN'] = df['SN'].astype(int)
        if year in df.columns:  # Check if the 'year' column was successfully added
            merged_dfs.append(df[['SN', year]])

    # Use reduce to merge all dataframes on 'SN' using an outer join, with an initial value in case of an empty list
    if len(merged_dfs):
        merged_df = reduce(lambda left, right: pd.merge(left, right, on='SN', how='outer'), merged_dfs)
    else:
        return pd.DataFrame(), pd.DataFrame()  # Return empty DataFrames if no files were processed

    # Keeping only the columns for 'SN' and the specified years
    valid_years = [year for year in years if year in merged_df.columns]
    required_columns = ['SN'] + valid_years
    merged_df = merged_df[required_columns]

    # Generating strong and swing DataFrames
    if valid_years:  # Only process if there are valid year columns
        conditions_won = [merged_df[year] == 'WON' for year in valid_years]
        strong_df = merged_df[reduce(lambda x, y: x & y, conditions_won)]
        swing_df = merged_df[reduce(lambda x, y: x | y, conditions_won)]
    else:
        strong_df = pd.DataFrame()  # Empty DataFrame if no valid year columns
        swing_df = pd.DataFrame()  # Empty DataFrame if no valid year columns

    return swing_df, strong_df


In [6]:

import os 


def create_strong_and_swing_and_dump_excel_files(ACS,excel_dir, output_dir, years, log_file_name):
    for i in range(1, ACS+1):
        try: 
            swing_df, strong_df = generate_swing_and_strong_booths_df(i, years, excel_dir)
            
            if not os.path.exists(output_dir):
                os.makedirs(output_dir)
            strong_file_path = f"{output_dir}/{i}_strong.xlsx"
            strong_df.to_excel(strong_file_path, index=False)
            
            swing_file_path = f"{output_dir}/{i}_swing.xlsx"
            swing_df.to_excel(swing_file_path, index=False)

        except Exception as exc: 
            with open(log_file_name, 'a') as f: 
                f.write(f"Error processing file: {i} - {exc}\n")
            continue
                

In [7]:
excel_dir = 'results/intermediate_tables/MH'
years = ['AE_2014', 'GE_2014', '2019'] 
ACS = 288
output_dir = 'results/strong_and_swing_booths/MH' 
log_file_name = 'logs/maharashtra_election_strong_swing_booths.txt'

create_strong_and_swing_and_dump_excel_files(ACS, excel_dir, output_dir, years, log_file_name)